In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import os
os.chdir('..')
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings

In [2]:
from langchain_chinese import (
    create_qa_chain,
    create_qa_toolkits,
    create_reason_agent,
)
from langchain_chinese.langgraph import create_tool_calling_executor

from langchain.schema import Document
query_document = Document(page_content="langchain_chinese 是为中国大语言模型优化的langchain模块")

from langchain_community.vectorstores import Chroma
db = Chroma.from_documents([query_document], ZhipuAIEmbeddings())

llm_zhipu = ChatZhipuAI(model="glm-4", streaming=True)

chain = create_qa_chain(llm_zhipu, db.as_retriever())
tools = create_qa_toolkits(qa_chain=chain)

In [3]:
from langchain.tools import tool
import random

# @tool(args_schema=WhereIsCatSchema)
@tool("WhereIsCatHidding")
def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底下", "在书架中", "在阳台"])

In [4]:
# from langchain_openai import ChatOpenAI
# model = ChatZhipuAI()
app = create_tool_calling_executor(llm_zhipu, [*tools, where_is_cat_hiding])

In [5]:
from langchain_core.messages import HumanMessage

inputs = [HumanMessage(content="请查询资料，告诉我langchain_chinese是什么？")]
for s in app.stream(inputs):
    print(s)
    print("----")

{'agent': AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8469012948968520491', 'function': {'arguments': '{"query":"langchain_chinese是什么"}', 'name': 'ask_document'}, 'type': 'function'}]})}
----


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'action': [ToolMessage(content='langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是用于提高语言模型的处理能力和效率，使其更适合中文环境下的应用需求。', tool_call_id='call_8469012948968520491')]}
----
{'agent': AIMessage(content='根据我的查询结果，langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是用于提高语言模型的处理能力和效率，使其更适合中文环境下的应用需求。')}
----
{'__end__': [HumanMessage(content='请查询资料，告诉我langchain_chinese是什么？'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8469012948968520491', 'function': {'arguments': '{"query":"langchain_chinese是什么"}', 'name': 'ask_document'}, 'type': 'function'}]}), ToolMessage(content='langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是用于提高语言模型的处理能力和效率，使其更适合中文环境下的应用需求。', tool_call_id='call_8469012948968520491'), AIMessage(content='根据我的查询结果，langchain_chinese 是专门针对中国大语言模型进行优化设计的langchain模块。这个模块可能是用于提高语言模型的处理能力和效率，使其更适合中文环境下的应用需求。')]}
----


In [6]:
from langchain_core.messages import HumanMessage

inputs = [HumanMessage(content="请查询资料，告诉我langchain_chinese是什么？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-chinese-gSQlHcwW-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


__{}


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'chunk': [ToolMessage(content='langchain_chinese 是专门为中国大语言模型优化的langchain模块。这个模块可能是用于改进语言模型的处理能力，比如在理解中文语境、文化背景以及语法结构等方面，以便更好地服务于中文用户的需求。', tool_call_id='call_8469018240368352891')]}
{'input': [HumanMessage(content='请查询资料，告诉我langchain_chinese是什么？'), AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8469018240368352891', 'function': {'arguments': '{"query":"langchain_chinese是什么"}', 'name': 'ask_document'}, 'type': 'function'}]})], 'output': [ToolMessage(content='langchain_chinese 是专门为中国大语言模型优化的langchain模块。这个模块可能是用于改进语言模型的处理能力，比如在理解中文语境、文化背景以及语法结构等方面，以便更好地服务于中文用户的需求。', tool_call_id='call_8469018240368352891')]}
根据_我的_查询_结果_，_lang_chain__ch_inese_ 是_专门_为中国_大_语言_模型_优_化的_lang_chain_模块_。_这个_模块_可能是_用于_改进_语言_模型的_处理_能力_，_比如_在_理解_中文_语境_、_文化_背景_以及_语法_结构_等方面_，_以便_更好地_服务于_中文_用户_的需求_。__

In [19]:
inputs = [HumanMessage(content="猫在哪里？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__{}
{'chunk': [ToolMessage(content='在书架中', tool_call_id='call_8469008722720615293')]}
{'input': [HumanMessage(content='猫在哪里？'), AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8469008722720615293', 'function': {'arguments': '{"idea":"猫在哪里？"}', 'name': 'WhereIsCatHidding'}, 'type': 'function'}]})], 'output': [ToolMessage(content='在书架中', tool_call_id='call_8469008722720615293')]}
根据_我的_调查_，_猫_喜欢_躲在_狭窄_的空间_里_，_比如_书_架_、_衣柜_、_纸_箱_等等_。_所以_，_如果你_找不到_你的_猫_，_可以去_这些_地方_看看_。_根据_我的_调用_结果_，_猫_在_书_架_中_，_你可以_去_那里_找_找_看_。__

In [20]:
inputs =  [HumanMessage(content="霍金的生日是哪一天？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__{}


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'chunk': [ToolMessage(content='霍金的生日是1942年1月8日。', tool_call_id='call_8469018206008610065')]}
{'input': [HumanMessage(content='霍金的生日是哪一天？'), AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8469018206008610065', 'function': {'arguments': '{"query":"霍金的生日是哪一天？"}', 'name': 'ask_document'}, 'type': 'function'}]})], 'output': [ToolMessage(content='霍金的生日是1942年1月8日。', tool_call_id='call_8469018206008610065')]}
根据_资料_库_查询_，_霍_金的_生日_是_19_4_2_年_1_月_8_日_。__

In [21]:
inputs =  [HumanMessage(content="写一个关于程序员的笑话")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

为什么_程序员_总是_携带_电脑_？_因为他们_不想_被人_称为_“_裸_奔_者_”。__